In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# We opt to define all the global configuration settings through this dict
config = {"resampling_fs" : 22050, #expected resampling rate on HDF5 file creation
          "dst_path":"./choral_dst", # User Dataset path (if None, the default one will be used)
          "hdf5_filepath":"./choral_dst.hdf5"
         }

if not os.path.isdir('./debug'):
    os.mkdir('./debug')

<h1><center>Data Handling Methodology</center></h1>

For the first step of this task, we will work from the __Choral Singing Dataset__, which contains three choir songs composed of 4 groups of 4 singers (16 parts per song), for a total of 48 stems. The dataset can be downloaded from Zenodo at the following:

https://zenodo.org/record/2649950#.XlJRNy2ZPRY

If you happen to already have the dataset locally, please define it's local path through the ***dst_path*** variable. Otherwise, the cell below will take care of downloading it for you:

### 1. Dataset Download:

In [3]:
#This cell downloads the Mridangam Stroke Dataset in its entirety
import urllib.request
import zipfile
import os, sys,shutil

#Link for the dataset. If you already have the dataset locally, define its location through the config dict above 
url = 'https://zenodo.org/record/2649950/files/ChoralSingingDataset.zip'

if config["dst_path"] == None:
    main_data_dir = './choral_dst'
    if not os.path.exists(main_data_dir): #creating the directory if not exist
        os.mkdir(main_data_dir)
    print('Downloading Choral Singing Dataset Dataset...')
    foldername = url.split('/')[-1]
    urllib.request.urlretrieve(url,foldername)
    #Unzipping to a specific folder
    zip_ref = zipfile.ZipFile(foldername, 'r')
    zip_ref.extractall(main_data_dir)
    zip_ref.close()
    os.remove(foldername)#Removing the zip file
    print('Data downloaded and unzipped to: ',main_data_dir)
else:
    print('User Path Defined. Dataset Expected to be Found at the Following:\n'+config["dst_path"])
 

User Path Defined. Dataset Expected to be Found at the Following:
./choral_dst


### 2. Storing our Data in a Convenient Format

For the sake of simplicity and clarity, we will store our song stems in a **HDF5 file**, which will help us to manage the later steps a bit more easily. This hierarchical file format will allow us to store our data in an intuitive way as follow:

**[HDF5_File] -> [Set] -> [Part] -> [Song]**

Since our dataset is of *very* limited size, we will break it down as follow:

| Set   | Stems | Songs |
| :---:   | :---: | :---: |
| Train |    24   |    A, B   |
| Valid |   8    |   A, B    |
| Test  |16| C |

**Note:** By default, the Choral Singing Dataset follow the following naming convention:

*DatasetName_SongAcronym_PartName_PartNumber.wav*

In [5]:
import h5py
import librosa
import soundfile
import glob
import random
from tqdm import tqdm
from time import sleep
from pathlib import Path

# First check if HDF5 already exists:
hdf5_file = Path(config['hdf5_filepath'])
if not hdf5_file.is_file():
    tracks = dict()
    resampling_fs = config['resampling_fs']
    dst_path = './choral_dst' if (config['dst_path'] == None) else config['dst_path']

    # Get all the stem files
    dst_stems = glob.glob(dst_path+"/*.wav",recursive=False)
    # Extract only songs from name
    songs = set([os.path.splitext(os.path.basename(i))[0].split('_')[1] for i in dst_stems])
    # Take 2/3 of songs for train/valid
    train_val_songs = random.sample(songs, int(np.ceil(0.666*len(songs))))
    # Take the rest as test set
    test_song = list(set(songs) - set(train_val_songs))
    test_idx = [int(i) for i, track in enumerate(dst_stems) if [True for song in test_song if str('_'+song+'_') in track]]
    # Take one random group of singer per song for each song as validation set
    rand_part = random.randint(1,4)
    val_idx   = [int(i) for i, track in enumerate(dst_stems) if [True for song in train_val_songs if ((str('_'+song+'_') in track) and (str('_'+str(rand_part)+'.wav')) in track)]]
    # Deduce the train set from valid / test set
    train_idx = list(set(range(len(dst_stems))) -  (set(val_idx + test_idx)))

    # tracks['train'] = np.take(dst_stems, train_idx)
    tracks['train'] = np.take(dst_stems, train_idx)
    tracks['valid'] = np.take(dst_stems, val_idx)
    tracks['test'] = np.take(dst_stems, test_idx)

    print('Ready to create HDF5')

    h5file = h5py.File(config["hdf5_filepath"], "w")

    # Write stems to hdf5 file for train/valid/test partitions
    for curr_partition in ["train", "valid", "test"]:

        print("Writing " + curr_partition + " partition with "+str(len(tracks[curr_partition]))+" files...")

        stem_list = tracks[curr_partition]

        # Create group for set if needed
        if not str(curr_partition) in h5file:
            set_grp  = h5file.create_group(curr_partition)

        sleep(1)

        for track in tqdm(stem_list):

            filename = os.path.splitext(os.path.basename(track))[0].split('_')

            song = filename[1]
            part = ''.join(filename[2:4])

            # Create group for the Song if needed
            if not str(curr_partition+'/'+song) in h5file:
                song_grp  = set_grp.create_group(song)

            # Create group for the Part if needed
            if not str(curr_partition+'/'+song+'/'+part) in h5file:
                part_grp = h5file[str(curr_partition+'/'+song)]
                subgrp  = part_grp.create_group(part)

            # Once part groups / song subgroups are created, store file
            audio, s = librosa.load(track, sr=resampling_fs)
            subgrp = h5file[str(curr_partition+'/'+song+'/'+part)]
            subgrp.create_dataset("raw_wav",data=audio)

    print('Done Creating HDF5 at path: '+config['hdf5_filepath'])
    h5file.close()
else:
    print('HDF5 Already Exists. File Expected to be Found at the Following:\n'+config["hdf5_filepath"])

HDF5 Already Exists. File Expected to be Found at the Following:
./choral_dst.hdf5


### 3. Defining the Generator Function:

Instead of computing features on the entirety of each of these songs, we will create many song snippets (or frames), which will depict different possible musical settings (SAT, ATB, SB, etc.) out of the songs. This will allow us to cover all possible singer permutations on a per-frame basis, without having to compute and feed to the model substantial chunk of audio.

The next step would be to define the function that will generate these song frames for us. 

**Note:** This function will yield the song segments but won't render them to disk. We can proceed by computing everything on the fly and save ourselves some local memory space.

In [6]:
def batchGenerator(hdf5_filepath, batch_size, num_frames, partition='train',debug=False):
    
    dataset   = h5py.File(hdf5_filepath, "r")
    itCounter = 0

    itCounter = itCounter + 1
    
    # Choose random song and retrieve all available part for it
    randsong = random.choice(list(dataset[partition].keys())) # Get random song from dataset keys
    sources  = list(dataset[partition][randsong].keys())      # Get all available sources for song
    
    # Compute available parts per group
    part_per_group = dict({'soprano':None,'alto':None,'tenor':None,'bass':None})
    for group in part_per_group.keys():
        parts = [source[-1] for source in sources if group in source]
        part_per_group[group] = parts

    # Init the start/end values of song snippet
    startspl = 0
    endspl   = 0

    # Preping the output shape of the generator
    out_shapes = {'mix':np.zeros((batch_size, num_frames)),'num_sing':np.zeros((batch_size))}

    # Will create batch_size x mixes of unique singers permutations
    for i in range(batch_size):

        randsources = random.sample(part_per_group.keys(), random.randint(1,4)) # Randomize source pick

        # Get Start and End samples. Pick random part to calculate start/end spl
        randpart = random.choice(sources)
        startspl = random.randint(0,len(dataset[partition][randsong][randpart]['raw_wav'])-num_frames) # This assume that all stems are the same length
        endspl   = startspl+num_frames

        # Get Random Sources with part number: 
        randsources_with_part = ["{}{}".format(a, random.choice(part_per_group[a])) for a in randsources] # Concatenate strings for part name
        # Retrieve the chunks and store them in output shapes                                         
        for source in randsources_with_part:

            source_chunk = dataset[partition][randsong][source]['raw_wav'][startspl:endspl]              # Retrieve part's chunk
            out_shapes['mix'][i] = np.add(out_shapes['mix'][i],source_chunk)            # Add the chunk to the mix

        # Scale down the mix based off number of sources
        scaler = len(randsources_with_part)
        out_shapes['mix'][i] = (out_shapes['mix'][i]/scaler)
        out_shapes['num_sing'][i] = len(randsources_with_part)

    # if debug == True, pick one random mix from batch and synthesize it
    if debug==True:

        if partition=='train':
            rand_pick = random.randint(0,batch_size-1)
            debug_dir = './debug/it#'+str(itCounter)+'_batchpick#'+str(rand_pick)
            if not os.path.isdir(debug_dir):
                os.mkdir(debug_dir)
            soundfile.write(debug_dir+'/'+'mix_'+str(out_shapes['num_sing'][rand_pick])+'_singers_'+'.wav', out_shapes['mix'][rand_pick], resampling_fs, 'PCM_24')

batchGenerator(config['hdf5_filepath'], 16, 8192, partition='train')

As you can see, the generator function returns a dictionary of shape:<br>
```out_shapes = {'mix':np.zeros((batch_size, num_frames)),'num_sing':np.zeros((batch_size))}``` <br><br>where:
* the key ```'mix'``` holds ```<batch_size>``` number of snippet of size ```<num_frames>```
* the key ```'num_sing'``` holds the ground truth respective to each of these mixes

<br>The purpose of the generator function is to provide the time-domain snippets. For the pre-processing steps related to frequency-domain conversion, see the next section below:

<h1><center>Pre-processing Steps</center></h1>

The first step of our experiment entails training a deep neural network model 